**На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.**

In [35]:
from math import asin
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [40]:
df = pd.read_excel('gb_sem_8_hm.xlsx')

/Users/maestro/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning:

Unknown extension is not supported and will be removed



In [41]:
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [42]:
df.shape

(10000, 3)

In [43]:
df.USER_ID.nunique()

6324

In [44]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [45]:
df.shape

(7865, 3)

In [46]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [47]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [48]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [49]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [50]:
df_new.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [51]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [52]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [53]:
continious_result(control, treatment, column='REVENUE')

100%|███████████████████████████████████| 10000/10000 [00:03<00:00, 3151.40it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.162383,0.712143,"[-0.003, 0.323]",0.122477


In [54]:
control.describe()

,USER_ID,REVENUE
count,2390.000000,2390.000000
mean,5020.881590,0.196887
std,2904.850992,4.172201
min,2.000000,0.000000
25%,2517.250000,0.000000
50%,5012.500000,0.000000
75%,7616.000000,0.000000
max,9998.000000,196.010000


In [55]:
treatment.describe()

,USER_ID,REVENUE
count,2393.000000,2393.000000
mean,4967.943168,0.074935
std,2892.745368,0.858207
min,4.000000,0.000000
25%,2435.000000,0.000000
50%,4955.000000,0.000000
75%,7379.000000,0.000000
max,9995.000000,23.040000


## Ответ: Наблюдаем, что 0 входит в доверительный интервал, кроме того alpha и beta большие 5%. Это нам говорит об отсутствии статитстически значимых различий. Следовательно, рекомендуем отказаться от внедрения проверяемых изменений.